# Chapter 7: Function Decorators and Closures

The goal of this chapter is to explain how function decorators work. This requires understanding:

* How Python evaluates decorator syntax

* How Python decides whether a variable is local

* Why closures exist and how the work

* What problem is solved by `nonlocal`

This is helpful when tackling further decorator topics, such as:

* Implementing a well-behaved decorator

* Interesting decorators in the standard library

* Implementing a parameterised decorator

## Decorators 101

A decorator is a callable that takes another function as an argument (the decorated function).

The decorator may perform some processing with the decorated function, and returns it or replaces it with another function or callable object.

In [1]:
# Example 7-1. A decorater usually replaces a function with a different one.
def deco(func):
    def inner():
        print("running inner()")
    return inner

@deco
def target():
    print("running target()")
    
target()

running inner()


## When Python Executes Decorators

A key feature of decorators is that they run right after the decorated function is defined.

This is usually at *import time* (i.e. when a module is loaded by Python).

In [3]:
# Example 7-2. The registration.py module.
registery = []

def register(func):
    print("running register(%s)" % func)
    registery.append(func)
    return func

@register
def f1():
    print("running f1()")
    
@register
def f2():
    print("running f2()")
    
def f3():
    print("running f3()")
    
def main():
    print("running main()")
    print("registery ->", registery)
    f1()
    f2()
    f3()
    
if __name__ == "__main__":
    main()
    

running register(<function f1 at 0x1082d65e0>)
running register(<function f2 at 0x1082d68b0>)
running main()
registery -> [<function f1 at 0x1082d65e0>, <function f2 at 0x1082d68b0>]
running f1()
running f2()
running f3()


Note that `register` runs twice before any other function in the module. 

When `register` is called, it receives as an argument the function object being decorated.

After the module is loaded, the `registery` holds references to the two decorated functions.

These functions are only executed when explicitly called by `main`.

The above example is unusual in two ways:

1. The decorated function is defined in the same module as the decorated function. A real decorator is usually defined in one module and applied to functions in another.

2. The `register` decorator returns the same functions passed as an argument. In practice, most decorators define an inner function and return it.

If *registration.py* is imported (and not run as a script), the output is:

In [7]:
from examples import registration

running register(<function f1 at 0x10a6a21f0>)
running register(<function f2 at 0x10a7ec3a0>)


In [9]:
registration.registery

[<function examples.registration.f1()>, <function examples.registration.f2()>]

The main point here is to emphasize that function decorators are run as soon as the module is imported, but the decorated functions only run when explicitly invoked (*import time* vs. *runtime*).

## Decorator-Enhanced Strategy Pattern

Decorators can be used as a good enhancement to the [Strategy](https://en.wikipedia.org/wiki/Strategy_pattern) pattern presented in the previous chapter.

This solution has several advantages compared to those presented in the previous chapter (**p. 174**):

* Strategy functions don't need special names (i.e. drop `_promo` suffix).

* `@promotion` highlights the purpose of the decorated function and makes it easy to disable a promotion (just comment out the decorator).

* Discount strategies may be implemented in other modules, anywhere on the system, as long as the `@promotion` decorator is applied to them.

In [10]:
# Example 7-3. The promos list is filled by the promotion decorator
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points."""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


@promotion
def bulk_item(order):
    """10% discount for line orders with 20 or more units."""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


@promotion
def large_order(order):
    "7% discount on orders with at least 10 distinct items."
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10 :
        return order.total() * .07
    else:
        return 0


def best_promo(order):
    """Select best discount available."""
    return max(promo(order) for promo in promos)

## Variable Scope Rules

Decorators almost always return inner functions.

Code the uses inner functions almost always depends on **closures** to operate correctly.

To understand closures, we need to look at how variable scopes work in Python.

In [2]:
# Example 7-5
b = 6

def f(a):
    print(a)
    print(b)
    b = 9
    
f(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

When Python compiles the body of the function, it decides that `b` is a local variable (because it is assigned within the function). This is why the function in **example 7-5** fails.

Python assumes that variables declared in the body of a function are local.

If we want the interpreter to treat `b` as a global variable in spite of assignment within the function, we need to use the `global` declaration.

In [3]:
# Example 7-6
# Example 7-5
b = 6

def f(a):
    global b
    print(a)
    print(b)
    b = 9
    
f(3)

3
6


In [4]:
b

9

## Closures

A closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there.

Consider the examples below:

* We call `Averager()` or `make_averager()` to get a callable object `avg` that will update the historical series and calculate the current mean.

* In **example 7-8** `avg` is an instance of `Averager()` and it is easy to see where `avg` keeps the history (`self.series` instance attribute).

* In **example 7-9** `avg` is the inner function of the higher-order function `make_averager`. In this case it is not obvious where the historical series is kept. 

In [19]:
# Example 7-8
class Averager():
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [20]:
# Creates a callable instance
avg = Averager()
avg(10)

10.0

In [21]:
avg(11)

10.5

In [22]:
avg(12)

11.0

In [28]:
# Example 7-9
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [29]:
# Returns the averager object
avg = make_averager()
avg(10)

10.0

In [30]:
avg(11)

10.5

In [31]:
avg(12)

11.0

`series` is a local variable of `make_averager`, but when `avg(10)` is called, `make_averager` has already returned and its local scope is gone.

Within `averager`, `series` is a *free variable*. This means that the variable is not bound in the local scope.

However, `averager` is a *closure* with access to its enclosing scope, which includes `series`.

![](https://www.codevoila.com/cvuploads/images/201606/python_function_and_python_closure.png)

Inspecting the `__code__` attribute of the returned function reveals how Python keeps the names of local and free variables in the compiled body of the function.

In [33]:
avg.__code__.co_varnames

('new_value', 'total')

In [34]:
avg.__code__.co_freevars

('series',)

In [40]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## The `nonlocal` Declaration

In contrast, consider the following example which is meant to be a more efficient implementation of `averager` (saving only the numbers that are needed for calculation):

In [27]:
# Example 7-13. Broken implementation of make_averager.
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

This doesn't work because `count += 1` means the same as `count = count + 1` when `count` is a number or any immutable type.

With immutable types like numbers, strings, tuples, etc., rebinding them (as in `count = count + 1`) implicitly creates a new local variable `count`. It is no longer a free variable and therefore is not saved in the closure.

Since we are actually assigning to `count` in the body of `averager`, that makes it a local variable. The same applies to `total`.

To work around this we can use the `nonlocal` declaration, which lets us flag a variable as free variable even when it is assigned a new value within a function. If a new value is assigned to a `nonlocal` variable, the binding stored in the closure is changed.

In [41]:
# Example 7-14. Correct implementation of make_averager.
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

10.0

## Implementing a Simple Decorator

TODO